# Tutorial 11: Vector Workflows — Embeddings, Indexing, Similarity Search

Learn how to: 
- Generate embeddings with the Tensorus embedding agent
- Build or refresh FAISS-backed indexes
- Execute similarity and hybrid searches
- Inspect vector metrics and stats

The notebook relies on `tutorial_utils.py` for shared helpers and gracefully switches to demo mode when the API is offline.

In [ ]:
from tutorial_utils import ping_server, ensure_dataset, pretty_json
import numpy as np
import requests

API = "http://127.0.0.1:7860"
SERVER = ping_server(API)
print(f"\ud83d\udce1 Tensorus server available: {SERVER}")

DATASET = "vector_demo"
if SERVER:
    dataset_result = ensure_dataset(DATASET, API)
    print(pretty_json(dataset_result.payload))
else:
    print("Demo mode: assuming dataset exists.")


## Embed sample texts
When the server is available, this cell calls `/api/v1/vector/embed`. Otherwise it fabricates demo responses so you can still explore downstream logic.

In [ ]:
texts = [
    "Transformer attention weights",
    "Climate simulation tensor fields",
    "Financial risk tensor analytics",
]
embed_payload = {
    "texts": texts,
    "dataset_name": DATASET,
    "namespace": "demo",
    "tenant_id": "tutorial"
}

if SERVER:
    try:
        response = requests.post(f"{API}/api/v1/vector/embed", json=embed_payload, timeout=20)
        response.raise_for_status()
        embed_result = response.json()
    except requests.RequestException as exc:
        embed_result = {"error": str(exc), "demo_mode": True}
else:
    embed_result = {"demo_mode": True, "record_ids": [f"demo_vec_{i}" for i in range(len(texts))]}

print(pretty_json(embed_result))


## Build or refresh the vector index
FAISS-backed indexes are managed via `/api/v1/vector/index/build`.

In [ ]:
index_payload = {
    "dataset_name": DATASET,
    "index_type": "partitioned",
    "metric": "cosine",
    "num_partitions": 8
}

if SERVER:
    try:
        build_response = requests.post(f"{API}/api/v1/vector/index/build", json=index_payload, timeout=60)
        build_response.raise_for_status()
        index_result = build_response.json()
    except requests.RequestException as exc:
        index_result = {"error": str(exc), "demo_mode": True}
else:
    index_result = {"demo_mode": True, "index_stats": {"total_vectors": len(embed_payload["texts"]), "partitions": 8}}

print(pretty_json(index_result))


## Similarity search
Query the vector index for nearest neighbors.

In [ ]:
search_payload = {
    "query": "risk analytics",
    "dataset_name": DATASET,
    "k": 5,
    "namespace": "demo",
    "tenant_id": "tutorial"
}

if SERVER:
    try:
        search_response = requests.post(f"{API}/api/v1/vector/search", json=search_payload, timeout=20)
        search_response.raise_for_status()
        search_result = search_response.json()
    except requests.RequestException as exc:
        search_result = {"error": str(exc), "demo_mode": True}
else:
    search_result = {"demo_mode": True, "results": ["demo_vec_0", "demo_vec_1"], "search_time_ms": 42.0}

print(pretty_json(search_result))


## Metrics and stats
Inspect aggregated metrics to monitor index health.

In [ ]:
if SERVER:
    try:
        stats_resp = requests.get(f"{API}/api/v1/vector/stats/{DATASET}", timeout=10)
        stats_resp.raise_for_status()
        stats_payload = stats_resp.json()
    except requests.RequestException as exc:
        stats_payload = {"error": str(exc), "demo_mode": True}
else:
    stats_payload = {"demo_mode": True, "stats": {"namespaces": {"demo": {"count": len(texts)}}}}

print(pretty_json(stats_payload))
